## HTTP Servers and Clients

## Sending and receiving events with EventEmitters
EventEmitters are one of the core idioms of Node.js. Many of the core modules are EventEmitters, and also EventEmitters make an excellent skeleton to implement asynchronous programming. In this chapter, we'll work with the HTTPServer and HTTPClient objects. Both of them are subclasses of EventEmitter and rely on it to send events for each step of the HTTP protocol. 

In [1]:
// this defines a Pulser class

var events = require('events');
var util = require('util');

// Define the Pulser object
function Pulser() {
 events.EventEmitter.call(this);
}

util.inherits(Pulser, events.EventEmitter);

Pulser.prototype.start = function() {
 setInterval(() => {
 util.log('>>>> pulse');
 this.emit('pulse');
 util.log('<<<< pulse');
 }, 1000);
};

[Function]

This defines a Pulser class, which inherits from EventEmitter (using util.inherits). Its purpose is to send timed events, once a second, to any listeners. 

In [2]:
// Instantiate a Pulser object
var pulser = new Pulser();

// Handler function
pulser.on('pulse', () => {
 util.log('pulse received');
});

// Start it pulsing
pulser.start();

3 Jul 19:33:50 - >>>> pulse
3 Jul 19:33:50 - pulse received
3 Jul 19:33:50 - <<<< pulse
3 Jul 19:33:51 - >>>> pulse
3 Jul 19:33:51 - pulse received
3 Jul 19:33:51 - <<<< pulse
3 Jul 19:33:52 - >>>> pulse
3 Jul 19:33:52 - pulse received
3 Jul 19:33:52 - <<<< pulse
3 Jul 19:33:53 - >>>> pulse
3 Jul 19:33:53 - pulse received
3 Jul 19:33:53 - <<<< pulse
3 Jul 19:33:54 - >>>> pulse
3 Jul 19:33:54 - pulse received
3 Jul 19:33:54 - <<<< pulse
3 Jul 19:33:55 - >>>> pulse
3 Jul 19:33:55 - pulse received
3 Jul 19:33:55 - <<<< pulse


Here is an example from tutorials point...

In [1]:
// https://www.tutorialspoint.com/nodejs/nodejs_event_emitter.htm
    
var events = require('events');
var eventEmitter = new events.EventEmitter();

// listener #1
var listner1 = function listner1() {
   console.log('listner1 executed.');
}

// listener #2
var listner2 = function listner2() {
  console.log('listner2 executed.');
}

// Bind the connection event with the listner1 function
eventEmitter.addListener('connection', listner1);

// Bind the connection event with the listner2 function
eventEmitter.on('connection', listner2);

var eventListeners = require('events').EventEmitter.listenerCount(eventEmitter,'connection');
console.log(eventListeners + " Listner(s) listening to connection event");

// Fire the connection event 
eventEmitter.emit('connection');

// Remove the binding of listner1 function
eventEmitter.removeListener('connection', listner1);
console.log("Listner1 will not listen now.");

// Fire the connection event 
eventEmitter.emit('connection');

eventListeners = require('events').EventEmitter.listenerCount(eventEmitter,'connection');
console.log(eventListeners + " Listner(s) listening to connection event");

console.log("Program Ended.");

2 Listner(s) listening to connection event
listner1 executed.
listner2 executed.
Listner1 will not listen now.
listner2 executed.
1 Listner(s) listening to connection event
Program Ended.


The EventEmitter event has many names, and it can be anything that makes sense to you. You can define as many event names as you like. Event names are defined simply by calling .emit with the event name.

## HTTP server applications
In most cases, you'll be able to use an application framework such as Express that hides the HTTP protocol details, allowing the programmer to focus on business logic. We already saw a simple HTTP server application:

In [2]:
var http = require('http');
http.createServer((req, res) => {
 res.writeHead(200, {'Content-Type': 'text/plain'});
 res.end('Hello, World!\n');
}).listen(8124, '127.0.0.1');

console.log('Server running at http://127.0.0.1:8124');

Server running at http://127.0.0.1:8124


Now, let's look at something more interesting with different actions based on the URL.

In [3]:
var http = require('http');
var util = require('util');
var url = require('url');
var os = require('os');
var server = http.createServer();
server.on('request', (req, res) => {
 var requrl = url.parse(req.url, true);
 if (requrl.pathname === '/') {
 res.writeHead(200, {'Content-Type': 'text/html'});
 res.end(
`<html><head><title>Hello, world!</title></head>
<body><h1>Hello, world!</h1>
<p><a href='/osinfo'>OS Info</a></p>
</body></html>`);
 } else if (requrl.pathname === "/osinfo") {
 res.writeHead(200, {'Content-Type': 'text/html'});
 res.end(
`<html><head><title>Operating System Info</title></head>
<body><h1>Operating System Info</h1>
<table>
<tr><th>TMP Dir</th><td>${os.tmpDir()}</td></tr>
<tr><th>Host Name</th><td>${os.hostname()}</td></tr>
<tr><th>OS Type</th><td>${os.type()} ${os.platform()} ${os.arch()}
${os.release()}</td></tr>
<tr><th>Uptime</th><td>${os.uptime()} ${util.inspect(os.loadavg())}</
td></tr>
<tr><th>Memory</th><td>total: ${os.totalmem()} free: ${os.freemem()}</
td></tr>
<tr><th>CPU's</th><td><pre>${util.inspect(os.cpus())}</pre></td></tr>
<tr><th>Network</th><td><pre>${util.inspect(os.networkInterfaces())}</
pre></td></tr>
</table>
</body></html>`);
 } else {
 res.writeHead(404, {'Content-Type': 'text/plain'});
 res.end("bad URL "+ req.url);
 }
});
server.listen(8124);

console.log('listening to http://localhost:8124');

listening to http://localhost:8124


(node:11088) [DEP0022] DeprecationWarning: os.tmpDir() is deprecated. Use os.tmpdir() instead.


The real purpose of the template strings feature is supporting strings where we can easily substitute values directly into the string. Most other programming languages support this ability, and now JavaScript does too.

## HTTP Sniffer – listening to the HTTP conversation
The following code demonstrates a useful module that listens to all the HTTP Server events. It could be a useful debugging tool, which also demonstrates how HTTP server objects operate.

In [5]:
var util = require('util');
var url = require('url');
sniffOn = function(server) {
 server.on('request', (req, res) => {
 util.log('e_request');
 util.log(reqToString(req));
 });
 server.on('close', errno => { util.log('e_close errno='+ errno);
});
 server.on('checkContinue', (req, res) => {
 util.log('e_checkContinue');
 util.log(reqToString(req));
 res.writeContinue();
 });
 server.on('upgrade', (req, socket, head) => {
 util.log('e_upgrade');
 util.log(reqToString(req));
 });
 server.on('clientError', () => { util.log('e_clientError'); });
};
var reqToString = exports.reqToString = function(req) {
 var ret=`req ${req.method} ${req.httpVersion} ${req.url}` +'\n';
 ret += JSON.stringify(url.parse(req.url, true)) +'\n';
 var keys = Object.keys(req.headers);
 for (var i = 0, l = keys.length; i < l; i++) {
 var key = keys[i];
 ret += `${i} ${key}: ${req.headers[key]}` +'\n';
 }
 if (req.trailers)
 ret += req.trailers +'\n';
 return ret;
};

[Function: sniffOn]

In [6]:
var server = http.createServer();
//require('./httpsniffer').sniffOn(server);
sniffOn(server);
server.listen(8124);
console.log('listening to http://localhost:8124');

listening to http://localhost:8124
4 Jul 07:52:20 - e_request
4 Jul 07:52:20 - req GET 1.1 /
{"protocol":null,"slashes":null,"auth":null,"host":null,"port":null,"hostname":null,"hash":null,"search":"","query":{},"pathname":"/","path":"/","href":"/"}
0 host: localhost:8124
1 connection: keep-alive
2 upgrade-insecure-requests: 1
3 user-agent: Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36
4 accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
5 referer: http://localhost:8888/notebooks/4%20HTTP%20Servers%20and%20Clients.ipynb
6 accept-encoding: gzip, deflate, br
7 accept-language: en-GB,en-US;q=0.9,en;q=0.8
8 cookie: _xsrf=2|fa929ee8|64299ac4f1456df4a5cfdb56abc8903e|1529443044; username-localhost-8888="2|1:0|10:1530654392|23:username-localhost-8888|44:NGViYzI3OGM2NmIzNDYyZGExYmQzODIxZTdiZmJhMDI=|39c9654bfd0019af34afe0247389ccd32166f06a920d54852ec8dcea273a47e3"
[object Object]



## Web application frameworks
The HTTPServer object is very close to the HTTP protocol. It's better to abstract away the HTTP details and concentrate on your application.

## Getting started with Express
Express is perhaps the most popular Node.js web app framework. It's so popular that it's part of the MEAN Stack acronym. MEAN refers to MongoDB, ExpressJS, AngularJS, and Node.js. 

Back to the Python kernel so that we can run some command line actions.

In [3]:
! npm install express-generator@4.x

+ express-generator@4.16.0
added 8 packages in 6.95s


npm WARN saveError ENOENT: no such file or directory, open 'C:\Users\simon.garisch\package.json'
npm WARN enoent ENOENT: no such file or directory, open 'C:\Users\simon.garisch\package.json'
npm WARN simon.garisch No description
npm WARN simon.garisch No repository field.
npm WARN simon.garisch No README data
npm WARN simon.garisch No license field.



Many now recommend against installing modules globally. In the Twelve-Factor model, it's strongly recommended to not install
global dependencies, and that's what we're doing. https://expressjs.com/en/starter/generator.html

In [6]:
! npm install express-generator -g

C:\Anaconda2\express -> C:\Anaconda2\node_modules\express-generator\bin\express-cli.js
+ express-generator@4.16.0
added 10 packages in 2.392s


In [7]:
! npm install -g bower

C:\Anaconda2\bower -> C:\Anaconda2\node_modules\bower\bin\bower
+ bower@1.8.4
added 1 package in 13.102s


npm WARN deprecated bower@1.8.4: We don't recommend using Bower for new projects. Please consider Yarn and Webpack or Parcel. You can read how to migrate legacy project here: https://bower.io/blog/2017/how-to-migrate-away-from-bower/


In [6]:
! npm -h


Usage: npm <command>

where <command> is one of:
    access, adduser, bin, bugs, c, cache, completion, config,
    ddp, dedupe, deprecate, dist-tag, docs, doctor, edit,
    explore, get, help, help-search, i, init, install,
    install-test, it, link, list, ln, login, logout, ls,
    outdated, owner, pack, ping, prefix, profile, prune,
    publish, rb, rebuild, repo, restart, root, run, run-script,
    s, se, search, set, shrinkwrap, star, stars, start, stop, t,
    team, test, token, tst, un, uninstall, unpublish, unstar,
    up, update, v, version, view, whoami

npm <command> -h     quick help on <command>
npm -l           display full usage info
npm help <term>  search for help on <term>
npm help npm     involved overview

Specify configs in the ini-formatted file:
    C:\Users\simon.garisch\.npmrc
or on the command line via: npm <command> --key value
Config info can be viewed via: npm help config

npm@5.5.1 C:\Anaconda2\node_modules\npm


In [7]:
! express -h


  Usage: express [options] [dir]


  Options:

        --version        output the version number
    -e, --ejs            add ejs engine support
        --pug            add pug engine support
        --hbs            add handlebars engine support
    -H, --hogan          add hogan.js engine support
    -v, --view <engine>  add view <engine> support (dust|ejs|hbs|hjs|jade|pug|twig|vash) (defaults to jade)
        --no-view        use static html instead of view engine
    -c, --css <engine>   add stylesheet <engine> support (less|stylus|compass|sass) (defaults to plain css)
        --git            add .gitignore
    -f, --force          force on non-empty directory
    -h, --help           output usage information


In [1]:
! express --view=pug myapp


   create : myapp\
   create : myapp\public\
   create : myapp\public\javascripts\
   create : myapp\public\images\
   create : myapp\public\stylesheets\
   create : myapp\public\stylesheets\style.css
   create : myapp\routes\
   create : myapp\routes\index.js
   create : myapp\routes\users.js
   create : myapp\views\
   create : myapp\views\error.pug
   create : myapp\views\index.pug
   create : myapp\views\layout.pug
   create : myapp\app.js
   create : myapp\package.json
   create : myapp\bin\
   create : myapp\bin\www

   change directory:
     > cd myapp

   install dependencies:
     > npm install

   run the app:
     > SET DEBUG=myapp:* & npm start



In [11]:
! cd myapp

In [13]:
# confirm working directory
import os
os.getcwd()

'C:\\Users\\simon.garisch\\Desktop\\git\\node.js-web-development'

In [14]:
! echo %cd%

C:\Users\simon.garisch\Desktop\git\node.js-web-development


In [15]:
! dir

 Volume in drive C has no label.
 Volume Serial Number is 7895-D27C

 Directory of C:\Users\simon.garisch\Desktop\git\node.js-web-development

04/07/2018  08:14 AM    <DIR>          .
04/07/2018  08:14 AM    <DIR>          ..
03/07/2018  08:06 AM    <DIR>          .ipynb_checkpoints
03/07/2018  07:43 AM             2,746 1 About Node.js.ipynb
03/07/2018  07:43 AM            64,578 2 Setting up node.ipynb
03/07/2018  08:04 AM            27,377 3 Node.js Modules.ipynb
04/07/2018  08:14 AM            22,324 4 HTTP Servers and Clients.ipynb
04/07/2018  07:59 AM    <DIR>          fibonacci
03/07/2018  07:43 AM               100 module1.js
03/07/2018  07:43 AM               191 module2.js
04/07/2018  08:07 AM    <DIR>          myapp
03/07/2018  07:43 AM                33 nodeFile.js
03/07/2018  07:43 AM    <DIR>          parentDirModule
29/06/2018  08:28 AM                81 README.md
03/07/2018  07:43 AM                66 testModules.js
               9 File(s)        117,496 bytes
        

In [20]:
! cd myapp

In [21]:
! echo %cd%

C:\Users\simon.garisch\Desktop\git\node.js-web-development


In [16]:
! npm install

up to date in 1.123s


npm WARN saveError ENOENT: no such file or directory, open 'C:\Users\simon.garisch\package.json'
npm WARN enoent ENOENT: no such file or directory, open 'C:\Users\simon.garisch\package.json'
npm WARN simon.garisch No description
npm WARN simon.garisch No repository field.
npm WARN simon.garisch No README data
npm WARN simon.garisch No license field.



In [22]:
! SET DEBUG=myapp:* & npm start

npm ERR! path C:\Users\simon.garisch\package.json
npm ERR! code ENOENT
npm ERR! errno -4058
npm ERR! syscall open
npm ERR! enoent ENOENT: no such file or directory, open 'C:\Users\simon.garisch\package.json'
npm ERR! enoent This is related to npm not being able to find a file.
npm ERR! enoent 

npm ERR! A complete log of this run can be found in:
npm ERR!     C:\Users\simon.garisch\AppData\Roaming\npm-cache\_logs\2018-07-03T22_19_11_983Z-debug.log


It'll work if you run these from the command line rather than notebook cells. After running these:

Express Welcome to Express

## Walking through the default Express application
There are a number of files that were automatically created for us...

In [1]:
var testFolder = './myapp/';
var fs = require('fs');

fs.readdir(testFolder, (err, files) => {
  files.forEach(file => {
    console.log(file);
  });
})

app.js
bin
node_modules
package-lock.json
package.json
public
routes
views


## Calculating the Fibonacci sequence with an Express application


In [2]:
var fs = require('fs-extra') // got some extra file system functionality here

In [3]:
fs.removeSync('myappModified'); // remove the folder synchronously if it exists

In [ ]:
fs.copySync('myapp','myappModified'); // copy over myapp to myappModified so that we can work on it

Once the default express app has been copied we can start working on the modified version. Either that or manually copy it over. Now to modify:

In [5]:
var fs = require('fs-extra');
path = './myappModified/app.js';
console.log(fs.existsSync(path));

true


In [6]:
// https://www.devdungeon.com/content/working-files-javascript-nodejs
// https://stackoverflow.com/questions/14177087/replace-a-string-in-a-file-with-nodejs
// npm install replace-in-file

var replace = require('replace-in-file');

var options1 = { // our first replacement

  //Single file
  files: './myappModified/app.js',

  //Replacement to make (string or regex) 
  from: "var usersRouter = require('./routes/users');",
  to: "var fibonacci = require('./routes/fibonacci');",
};

var options2 = { // our second replacement
  files: './myappModified/app.js',
  from: "app.use('/users', usersRouter);",
  to: "app.use('/fibonacci', fibonacci);",
};

In [7]:
try {
  let changedFiles = replace.sync(options1);
}
catch (error) {
  console.error('Error occurred:', error);
}

In [8]:
try {
  let changedFiles = replace.sync(options2);
}
catch (error) {
  console.error('Error occurred:', error);
}

We should now have the app.js file changed as required. We'll write to a file sync called math.js

In [9]:
var fs = require('fs');
var code = `
var fibonacci = exports.fibonacci = function(n) {
 if (n === 1) return 1;
 else if (n === 2) return 1;
 else return fibonacci(n-1) + fibonacci(n-2);
}
`
fs.writeFileSync('./myappModified/math.js', code);

In the views directory, create a file named top.ejs:

In [10]:
var fs = require('fs');
var code = `
<html>
<head>
 <title><%= title %></title>
 <link rel='stylesheet' href='/stylesheets/style.css' />
</head>
<body>
 <h1><%= title %></h1>
 <div class='navbar'>
 <p><a href='/'>home</a>
 | <a href='/fibonacci'>Fibonacci's</a></p>
 </div>
`
fs.writeFileSync('./myappModified/views/top.ejs', code);

Another file named bottom.ejs:

In [11]:
var code = `
</body>
</html>
`
fs.writeFileSync('./myappModified/views/bottom.ejs', code);

In [12]:
var code = `
<% include top %>
<p>Welcome to the Math calculator</p>
<% include bottom %>
`
fs.writeFileSync('./myappModified/views/index.ejs', code);

In [14]:
var code = `
<% include top %>
<% if (typeof fiboval !== "undefined") { %>
 <p>Fibonacci for <%= fibonum %> is <%= fiboval %></p>
 <hr/>
<% } %>
<p>Enter a number to see its' Fibonacci number</p>
<form name='fibonacci' action='/fibonacci' method='get'>
<input type='text' name='fibonum' />
<input type='submit' value='Submit' />
</form>
<% include bottom %>
`
fs.writeFileSync('./myappModified/views/fibonacci.ejs', code);

In the routes directory, delete the user.js module. It is generated by the Express framework, but we will not use it in this application

In [ ]:
// check the ejs vs pug format... you may have to start the app template again pg 81